# HW2, Part 1: Real-world SQL #

## This assignment has been originally developed for [UC Berkeley CS186 course](http://www.cs186berkeley.net).  We use it for COMS4037 with their gracious permission. ##



In [8]:
import subprocess, sqlite3, csv, os

currentDB = "part1.db"

# for autograder!
# TODO: again this requires corresponding setup for grading!
if ( 'CS186GRADER' in os.environ ):
    currentDB = "part1.db"    

    
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

if not ( os.path.isfile( currentDB ) ):
    raise IOError( "DB File " + currentDB + " is not here!" )
db = sqlite3.connect( currentDB )
db.text_factory = str
db.row_factory = dict_factory
cur = db.cursor()

In [9]:
# Q1
# Question: for all companies whose status column contains 'acquired', show
# the associated acquisition price_amount (if there is no acquisition record, price_amount
# should be returned as NULL)
# Note: ignore duplicate data issues 
# Output: company_name, price_amount

def q1():
    db.executescript("""
        DROP VIEW IF EXISTS q1;

        CREATE VIEW q1 AS
        -- TODO
        SELECT C.company_name AS company_name, A.price_amount AS price_amount
        FROM companies C
        LEFT JOIN acquisitions A 
        ON LOWER(c.company_name) = LOWER(a.company_name)
        WHERE LOWER(c.status) LIKE 'acquired'
        ORDER BY C.company_name
        ;
    """)
    return
q1()
cur.execute("SELECT * FROM (SELECT company_name, price_amount FROM q1)")

print cur.fetchall()

[{'price_amount': '', 'company_name': '#waywire'}, {'price_amount': 18000000, 'company_name': '(RED)'}, {'price_amount': '', 'company_name': '1000 Markets'}, {'price_amount': '', 'company_name': '1000memories'}, {'price_amount': '', 'company_name': '100Plus'}, {'price_amount': 500000000, 'company_name': '1010data'}, {'price_amount': '', 'company_name': '12Society'}, {'price_amount': '', 'company_name': '13th Lab'}, {'price_amount': 43000000, 'company_name': '170 Systems'}, {'price_amount': '', 'company_name': '1CLICK'}, {'price_amount': 20000000, 'company_name': '280 North'}, {'price_amount': 20000000, 'company_name': '280 North'}, {'price_amount': '', 'company_name': '29West'}, {'price_amount': '', 'company_name': '2Web Technologies'}, {'price_amount': 475000000, 'company_name': '2Wire'}, {'price_amount': '', 'company_name': '2lemetry'}, {'price_amount': 287500000, 'company_name': '2nd Story Software, Inc.'}, {'price_amount': '', 'company_name': '3 day Blinds'}, {'price_amount': '', '

In [10]:
# Q2
# Question: How many startups, according this data, were founded (founded_at column) between 
# 2012 and 2014, inclusive (on or after 2012-01-01, and before or on 2014-12-31)?
# Output: a number (column name doesn't matter)
# Hint: time comparison will look like <= '1990-01-12'
# Note: ignore duplicate data issues 

def q2():
    db.executescript("""
        DROP VIEW IF EXISTS q2;

        CREATE VIEW q2 AS
        -- TODO
            SELECT COUNT(DISTINCT company_name) FROM companies WHERE founded_at >= '2012-01-01' AND founded_at <= '2014-12-31';
    """)
    return

q2() # called for your convenience

cur.execute("SELECT * FROM q2")
print cur.fetchall()

[{'COUNT(DISTINCT company_name)': 16179}]


In [6]:
# Q3
# Question: What is/are the state(s) that has/have the largest number(s) of startups  in the  
#    "Security" market (i.e. market column contains the word "Security")? (Plural due to ties)
# Output: the state(s) and the corresponding number(s), column named "state" and "total" respectively.
#         Note that the test script relies on the naming of output columns.

# Notes:
# - If the output is tied, then display all (i.e. if both CA and NY have, say 1000 startups, 
#   then display both states)
# - You might want to look into the keyword "LIKE"
# - The state must be valid (i.e. not "")
# Note: ignore duplicate data  issues 

def q3():
    db.executescript("""
        DROP VIEW IF EXISTS q3;
        
        CREATE VIEW q3 AS        
        -- TODO
        SELECT c.state AS state, COUNT(c.company_name) AS total--, count(c1.company_name)
        FROM companies AS c
        WHERE LOWER(c.market) LIKE '%securit%' AND c.state NOT NULL AND c.state != ''
        GROUP BY c.state
        HAVING COUNT(c.company_name) =
        (SELECT MAX(cnt) FROM (SELECT c1.state, COUNT(c1.company_name) AS cnt--, count(c1.company_name)
        FROM companies AS c1
        WHERE LOWER(c1.market) LIKE '%securit%' AND c1.state NOT NULL AND c1.state != ''
        GROUP BY c1.state));
    """)
    return

q3()
cur.execute("SELECT * FROM q3")
print cur.fetchall()

[{'state': 'CA', 'total': 216}]


In [11]:
# Q4
# Question: which cities have a larger number of acquirers than startups?

# Note 
# - a startup could be counted as an acquirer, 
#   but it should only be counted at most once in each category
#   and the data is not very clean (i.e. duplicates in company_name entry)

def q4():
    db.executescript("""
        DROP VIEW IF EXISTS q4;
        CREATE VIEW q4 AS
        --TODO  
        SELECT DISTINCT a_city FROM(
            SELECT 
            startups.city AS s_city, startups.count AS s_count, acquirers.city AS a_city, acquirers.count AS a_count FROM 
                (SELECT city, COUNT(c_name) AS count FROM 
                    ( 
                    --Disregard any duplicates
                    SELECT company_name AS c_name, city, market
                    FROM companies
                    WHERE city NOT NULL AND city NOT like ''
                    EXCEPT
                    --Disregard any duplicates
                    SELECT acquirer_name, acquirer_city, acquirer_market
                    FROM acquisitions 
                    WHERE acquirer_city NOT NULL AND acquirer_city NOT LIKE ''
                    ) 
                GROUP BY city) AS startups,
                (SELECT city, COUNT(a_name) AS count FROM 
                    (
                    --Disregard any duplicates
                    SELECT acquirer_name  AS a_name, acquirer_city AS city, acquirer_market AS market
                    FROM acquisitions 
                    WHERE acquirer_city NOT NULL AND acquirer_city NOT  LIKE ''
                    )
                GROUP BY city) AS acquirers
                WHERE startups.city = acquirers.city OR 
                startups.city IS NULL OR acquirers.city IS NULL
        )
        WHERE s_count < a_count;
    """)
    return
q4()
cur.execute("SELECT * FROM q4")
print cur.fetchall()

[{'a_city': 'Albany'}, {'a_city': 'Aliso Viejo'}, {'a_city': 'Alviso'}, {'a_city': 'Antwerpen'}, {'a_city': 'Armonk'}, {'a_city': 'Australia'}, {'a_city': 'Bad Vilbel'}, {'a_city': 'Bagneux'}, {'a_city': 'Ban Nong Kong Chak'}, {'a_city': 'Basel'}, {'a_city': 'Basking Ridge'}, {'a_city': 'Beloit'}, {'a_city': 'Beltsville'}, {'a_city': 'Berkeley Heights'}, {'a_city': 'Bloomingdale'}, {'a_city': 'Bonn'}, {'a_city': 'Bossier City'}, {'a_city': 'Braintree'}, {'a_city': 'Brookfield'}, {'a_city': 'Bryanston'}, {'a_city': 'Buckhead'}, {'a_city': 'Buena Park'}, {'a_city': 'Camden'}, {'a_city': 'Canonsburg'}, {'a_city': 'Carrollton'}, {'a_city': 'Carson'}, {'a_city': 'Cedar Valley'}, {'a_city': 'Central'}, {'a_city': 'Chelmsford'}, {'a_city': 'Chesterfield'}, {'a_city': 'Chicago Heights'}, {'a_city': 'Coloma'}, {'a_city': 'Corning'}, {'a_city': 'Daly City'}, {'a_city': 'Danbury'}, {'a_city': 'Darlington'}, {'a_city': 'Deerfield'}, {'a_city': 'Des Plaines'}, {'a_city': 'Downers Grove'}, {'a_city'

In [27]:
q1()
q2()
q3()
#q4()
! ./test.sh

diff: ~cs186/sp16/hw2/part1/expected_output/q1.csv: No such file or directory
ERROR q1 output differed! See diffs/q1.csv
diff: ~cs186/sp16/hw2/part1/expected_output/q2.csv: No such file or directory
ERROR q2 output differed! See diffs/q2.csv
diff: ~cs186/sp16/hw2/part1/expected_output/q3.csv: No such file or directory
ERROR q3 output differed! See diffs/q3.csv
Error: no such column: acquirer_city
ERROR q4! See your_output/q4.csv
